<a href="https://colab.research.google.com/drive/14925HTxumWESLpXKyJsENKd5NY32VNFk?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>

### 📚 What is Basic RAG?

Basic RAG is the standard, straightforward implementation of **Retrieval-Augmented Generation**. It involves retrieving relevant information from a knowledge base in response to a query, then using this information to generate an answer using a language model.

### ❓ Why we need RAG?

1. Combines the broad knowledge of language models with specific, up-to-date information
2. Improves accuracy of responses by grounding them in retrieved facts
3. Reduces hallucinations common in standalone language models
4. Allows for easy updates to the knowledge base without retraining the entire model



# Install required libraries

In [ ]:
!pip install -q -U \
     Sentence-transformers==3.0.1 \
     langchain==0.3.19 \
     langchain-groq==0.2.4 \
     langchain-community==0.3.18 \
     langchain-huggingface==0.1.2 \
     einops==0.8.1 \
     faiss_cpu==1.10.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.0/136.0 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 107.9 MB/s eta 0:00:00


# Import related libraries related to Langchain, HuggingfaceEmbedding

In [ ]:
# Import Libraries
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [ ]:
import getpass
import os

#### Provide a Groq API key. You can create one to access free open-source models at the following link.

[Groq API Creation Link](https://console.groq.com/keys)




In [ ]:
os.environ["GROQ_API_KEY"] = getpass.getpass()

··········


#### Provide Huggingface API Key. You can create Huggingface API key at following lin

[Huggingface API Creation Link](https://huggingface.co/settings/tokens)




In [ ]:
os.environ["HF_TOKEN"] = getpass.getpass()

··········


In [ ]:
# Helper function for printing docs
def pretty_print_docs(docs):
    # Iterate through each document and format the output
    for i, d in enumerate(docs):
        print(f"{'-' * 50}\nDocument {i + 1}:")
        print(f"Content:\n{d.page_content}\n")
        print("Metadata:")
        for key, value in d.metadata.items():
            print(f"  {key}: {value}")
    print(f"{'-' * 50}")  # Final separator for clarity

# Example usage
# Assuming `docs` is a list of Document objects


In [ ]:
%pip install -qU langchain-community pypdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.6/326.6 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.2/471.2 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.19 requires langchain-core<1.0.0,>=0.3.35, but you have langchain-core 1.0.4 which is incompatible.
langchain 0.3.19 requires langchain-text-splitters<1.0.0,>=0.3.6, but you have langchain-text-splitters 1.0.0 which is incompatible.
langchain-huggingface 0.1.2 requires langchain-core<0.4.0,>=0.3.15, but you have langchain-core 1.0.4 which is incompatible.
langchain-groq 0.2.4 requires langchain-core<0.4.0,>=0.3.33

# Basic RAG in Action

In [ ]:
# Import necessary libraries
from langchain_community.document_loaders import WebBaseLoader, PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load the PDF
file_path = "/content/attention is all you need.pdf"
loader = PyPDFLoader(file_path)
documents = loader.load()  # <-- load the PDF into documents

# Split documents into chunks of 500 characters with 100 characters overlap
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
texts = text_splitter.split_documents(documents)

# Add unique IDs to each text chunk
for idx, text in enumerate(texts):
    text.metadata["id"] = idx


print(documents)

[Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-04-10T21:11:43+00:00', 'author': '', 'keywords': '', 'moddate': '2024-04-10T21:11:43+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': '/content/attention is all you need.pdf', 'total_pages': 15, 'page': 0, 'page_label': '1'}, page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.edu\nŁukasz Kaise

In [ ]:
# Create embeddings for the text chunks
embedding = HuggingFaceEmbeddings(model_name="nomic-ai/nomic-embed-text-v1.5", model_kwargs = {'trust_remote_code': True})

# Initialize a FAISS (Vector Store) retriever with the text embeddings
retriever = FAISS.from_documents(texts, embedding).as_retriever(search_kwargs={"k": 20})

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/255 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/140 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

In [ ]:
# Import the RetrievalQA chain for question-answering tasks
from langchain.chains import RetrievalQA

# Define a query and retrieve relevant documents
query = "give me summary of this?"

llm = ChatGroq(
    model="openai/gpt-oss-120b",
    temperature=0.9
)

# Create a RetrievalQA chain using the language model and the retriever
chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

# Invoke the chain with a specific query to get a response
reponse = chain.invoke(query)

# Print the result of the response
print(reponse["result"])

**Summary**

The passage discusses the **Transformer** architecture, a neural model for sequence‑to‑sequence tasks that relies entirely on **self‑attention** mechanisms instead of recurrence or convolution.

| Component | Key Points |
|-----------|------------|
| **Self‑Attention** | • Relates every position in a sequence to every other position.<br>• Computed as `Attention(Q, K, V) = softmax(Q Kᵀ / √dₖ) V`.<br>• Two main variants: additive attention and scaled dot‑product attention (the latter used in the Transformer). |
| **Complexity & Parallelism** | • Self‑attention has per‑layer complexity **O(n²·d)** (n = sequence length, d = model dimension) but requires **O(1)** sequential operations, giving the shortest possible path length for long‑range dependencies.<br>• Recurrent layers are **O(n·d²)** with O(n) sequential steps and path length O(n).<br>• Convolutional layers are **O(k·n·d²)** (k = kernel size) with O(1) sequential steps and path length O(logₖ n). |
| **Positional Encodin

In [ ]:
# Define a query and retrieve relevant documents
query = "what is encoder?"

# Invoke the chain with a specific query to get a response
reponse = chain.invoke(query)

# Print the result of the response
print(reponse["result"])

In the Transformer architecture the **encoder** is the part of the model that first processes the input sequence and turns it into a set of continuous‑valued representations that can later be used by the decoder.

Key characteristics of the encoder (as described in the paper excerpt):

| Aspect | Description |
|--------|-------------|
| **Structure** | It consists of **N = 6 identical layers** stacked on top of one another. |
| **Sub‑layers per layer** | Each layer has two sub‑layers: <br>1. A **multi‑head self‑attention** mechanism. <br>2. A **position‑wise feed‑forward network** (two linear transformations with a ReLU in between). |
| **Self‑attention** | In every self‑attention sub‑layer, the **queries, keys, and values all come from the same source** – the output of the previous encoder layer (or the input embeddings for the first layer). This lets each position in the sequence attend to every other position in the same layer. |
| **Residual connections & normalization** | Around e